# Chapter 5 pandas 시작하기

- pandas는 아래의 도구들과 같이 사용하는 경우가 흔하다.
    - 산술 계산 도구인 NumPy와 SciPy
    - 분석 라이브러리인 statsmodels와 scikit-learn
    - 시각화 도구인 matplotlib  
- pandas는 for 문을 사용하지 않고 데이터를 처리한다거나 배열 기반의 함수를 제공하는 등 NumPy의 배열 기반 계산 스타일을 많이 차용했다.
- pandas의 import 컨벤션은 `import pandas as pd`
- E또한 `Series`와 `DataFrame`은 로컬 네임스페이스로 임포트하는 것이 컨벤션

In [1]:
import pandas as pd
from pandas import Series, DataFrame

## 5.1 pandas 자료구조 소개

- pandas에 대해 알아보려면 Series와 DataFrame, 이 두 가지 자료구조에 익숙해질 필요가 있다.

### 5.1.1 Series

- Series는 일련의 객체를 담을 수 있는 1차원 배열 같은 자료구조다 (어떤 NumPy 자료형이라도 담을 수 있다).
- $색인^{\text{index}}$이라고 하는 배열의 데이터와 연관된 `이름`을 가지고 있다.
- 가장 간단한 Series 객체는 배열 데이터로부터 생성할 수 있다.

In [2]:
obj = pd.Series([4,7,-5,3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

In [7]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [8]:
obj.index  # range(4)와 같다.

RangeIndex(start=0, stop=4, step=1)

> 별도로 index를 지정해 줄 수 있다.

In [9]:
obj2 = pd.Series([4,7,-5,3], index = ['d','b','a','c'])
obj2

d    4
b    7
a   -5
c    3
dtype: int64

In [12]:
obj2.index

Index(['d', 'b', 'a', 'c'], dtype='object')

> 다음과 같은 활용 방안도 있다.

In [22]:
import numpy as np

print('''
obj2['a'] : {0} \n

------------------

obj2[['c','a','d']] 

{1} 

------------------

obj2[obj2 > 0] 

{2}
    
------------------
    
np.exp(obj2) 

{3}'''.format(obj2['a'], \
              obj2[['c','a','d']], \
              obj2[obj2 > 0], \
              np.exp(obj2)))


obj2['a'] : -5 


------------------

obj2[['c','a','d']] 

c    3
a   -5
d    4
dtype: int64 

------------------

obj2[obj2 > 0] 

d    4
b    7
c    3
dtype: int64
    
------------------
    
np.exp(obj2) 

d      54.598150
b    1096.633158
a       0.006738
c      20.085537
dtype: float64


> Series는 색인값에 데이터값을 매핑하고 있으므로 파이썬의 사전형과 비슷하다.  
> Series 객체는 파이썬의 사전형을 인자로 받아야 하는 많은 함수에서 사전형을 대체하여 사용할 수 있다.

In [23]:
sdata = {'Ohio' : 35000, 'Texas' : 71000, 'Oregon' : 16000, 'Utah' : 5000}

obj3 = pd.Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

> 색인을 직접 지정하고 싶다면 원하는 순서대로 색인을 직접 넘겨줄 수도 있다.

In [25]:
states = ['California','Ohio','Oregon','Texas']

obj4 = pd.Series(sdata, index = states)
obj4 

California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
dtype: float64

> 위 결과에서 California index에 해당하는 값이 없으므로 $\text{NAN}^{\text{not a number}}$으로 표시된다.  
> 'Utah'는 states에 포함되어 있지 않으므로 실행 결과에서 빠지게 된다.

> pandas의 isnull과 notnull 함수는 누락된 데이터를 찾을 때 사용된다.

In [32]:
pd.isnull(obj4)

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

In [27]:
pd.notnull(obj4)

California    False
Ohio           True
Oregon         True
Texas          True
dtype: bool

In [28]:
# 해당 메서드는 Series 객체의 인스턴트 메서드로도 존재한다.
obj4.isnull()

California     True
Ohio          False
Oregon        False
Texas         False
dtype: bool

> Series의 유용한 기능은 산술 연산에서 색인과 라벨로 자동 정렬하는 것이다.  
> 데이터베이스를 사용해본 경험이 있다면 join 연산과 비슷하다고 여겨질 것이다.

In [33]:
obj3 + obj4

California         NaN
Ohio           70000.0
Oregon         32000.0
Texas         142000.0
Utah               NaN
dtype: float64

> Series 객체와 Series의 색인은 모두 name 속성이 있음. 이 속성은 pandas의 핵심 기능과 밀접한 관련이 있다.

In [35]:
obj4.name = 'population'
obj4.index.name = 'state'

obj4

state
California        NaN
Ohio          35000.0
Oregon        16000.0
Texas         71000.0
Name: population, dtype: float64

> Series의 색인은 대입하여 변경할 수 있다.

In [36]:
obj.index = ['Bob','Steve','Jeff','Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

### 5.1.2 DataFrame
- DataFrame은 표 같은 스프레드시트 형식의 자료구조
- 여러 개의 컬럼이 있으며 각 컬럼은 서로 다른 종류의 값 (숫자, 문자열, 불리언 등)을 담을 수 있음
- DataFrame은 로우와 컬럼에 대한 색인을 가지고 있음. 색인의 모양이 같은 Series 객체를 담고 있는 파이썬 사전으로 생각하면 편하다.
- 물리적으로 DataFrame은 2차원이지만 계층적 색인을 이용해서 좀 더 고차원의 데이터를 표현할 수 있음
- DataFrame 객체를 생성하는 가장 흔한 방법은 같은 길이의 리스트에 담긴 사전을 이용하거나 NumPy 배열을 이용하는 것이다.

In [38]:
data = {'state' : ['Ohio','Ohio','Ohio','Nevada','Nevada','Nevada'],
        'year' : [2000,2001,2002,2001,2002,2003],
        'pop' : [1.5,1.7,3.6,2.4,2.9,3.2]}
frame = pd.DataFrame(data)
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9
5,Nevada,2003,3.2


> 원하는 순서대로 columns를 지정하면 원하는 순서를 가진 DataFrame 객체가 `생성`된다.

In [40]:
pd.DataFrame(data, columns = ['year','state','pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9
5,2003,Nevada,3.2


> Series와 마찬가지로 사전에 없는 값을 넘기면 결측치로 저장된다.

In [47]:
frame2 = pd.DataFrame(data, 
                      columns = ['year','state','pop','debt'],
                      index = ['one','two','three','four','five','six'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN
six,2003,Nevada,3.2,NaN


In [50]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

> DataFrame의 컬럼은 Series처럼 `사전 형식의 표기법`으로 접근하거나 `속성 형식`으로 접근할 수 있다.

In [51]:
frame2['state']   # 사전 형식의 표기법으로 컬럼에 접근하는 방법

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
six      Nevada
Name: state, dtype: object

In [53]:
frame2.year # 속성 형식으로 컬럼에 접근하는 방법

one      2000
two      2001
three    2002
four     2001
five     2002
six      2003
Name: year, dtype: int64

> 로우는 위치나 loc 속성을 이용해서 이름을 통해 접근할 수 있다.

In [58]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

> 컬럼은 스칼라값이나 배열의 값으로 대입 가능하다.  
> 리스트나 배열을 컬럼에 대입할 때는 대입하려는 값의 길이가 DataFrame의 크기와 `동일`해야 한다.

In [63]:
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5
six,2003,Nevada,3.2,16.5


In [66]:
frame2['debt'] = np.arange(6.)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0.0
two,2001,Ohio,1.7,1.0
three,2002,Ohio,3.6,2.0
four,2001,Nevada,2.4,3.0
five,2002,Nevada,2.9,4.0
six,2003,Nevada,3.2,5.0


> Series를 컬럼에 대입하는 경우 DataFrame의 색인에 따라 값이 대입되며 존재하지 않는 색인에는 결측치가 대입된다.

In [70]:
val = pd.Series([-1.2, -1.5, -1.7], index = ['two','four','five'])

frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


> 존재하지 않는 컬럼을 대입하면 새로운 컬럼을 생성한다.  
> del 예약어를 사용해서 컬럼을 삭제할 수 있다.

In [72]:
frame2['eastern'] = frame2['state'] == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False
six,2003,Nevada,3.2,NaN,False


In [73]:
del frame2['eastern']
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7
six,2003,Nevada,3.2,NaN


> 중첩된 사전을 이용해서 데이터를 생성할 수 있다.  
> 중첩된 사전의 `바깥에 있는 키`는 `컬럼`이 되고 `안에 있는 키`는 `로우`가 된다.

In [74]:
pop = {'Nevada' : {2001 : 2.4, 2002 : 2.9},
       'Ohio' : {2000 : 1.5, 2001 : 1.7, 2002 : 3.6}}

frame3 = pd.DataFrame(pop)
frame3

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


> 만일 데이터 프레임의 색인(index)과 컬럼(columns)에 name 속성을 지정했다면 이 역시 함께 출력됨

In [76]:
frame3.index.name = 'year'; frame3.columns.name = 'state'
frame3

state,Nevada,Ohio
year,,
2001,2.4,1.7
2002,2.9,3.6
2000,NaN,1.5


> Series와 유사하게 values 속성은 DataFrame에 저장된 데이터를 2차원 배열로 반환함

In [77]:
frame3.values

array([[2.4, 1.7],
       [2.9, 3.6],
       [nan, 1.5]])

#### DataFrame 생성자에 넘길 수 있는 자료형 목록

<details>
<summary>DataFrame 생성을 위한 입력 데이터의 종류</summary>
<div markdown="1">

|형|설명|
|:--|:--|
|2차원 ndarray|데이터를 담고 있는 행렬. 선택적으로 행(로우)과 열(컬럼)의 이름을 전달할 수 있다.|
|배열, 리스트, 튜플의 사전|사전의 모든 항목은 같은 길이를 가져야 하며, 각 항목의 내용이 DataFrame의 컬럼이 된다.|
|NumPy의 구조화 배열|배열의 사전과 같은 방식으로 취급된다.|
|Series의 사전|Series의 각 값이 컬럼이 된다. 명시적으로 색인을 넘겨주지 않으면 각 Series의 색인이 하나로 합쳐져서 로우의 색인이 된다.|
|사전의 사전|내부에 있는 사전이 컬럼이 된다. 키값은 'Series의 사전'과 마찬가지로 합쳐져서 로우의 색인이 된다.|
|사전이나 Series의 리스트|리스트의 각 항목이 DataFrame의 로우가 된다. 합쳐진 사전의 키값이나 Series의 색인이 DAtaFrame의 컬럼 이름이 된다.|
|리스트나 튜플의 리스트|'2차원 ndarray'의 경우와 같은 방식으로 취급된다.|
|다른 DataFrame|색인을 따로 지정하지 않으면 DataFrame의 색인이 그대로 사용된다.|
|NumPy MaskedArray|'2차원 ndarray'의 경우와 같은 방식으로 취급되지만 마스크값은 반환되는 DataFrame에서 NA값이 된다.|
    
</div>
</details>

### 5.1.3 색인 객체 
- pandas의 색인 객체는 표 형식의 데이터에서 `각 로우와 컬럼에 대한 이름`과 다른 메타데이터 (`축의 이름 등`)을 저장하는 `객체`다.
- Series나 DataFrame 객체를 생성할 때 사용되는 배열이나 다른 순차적인 이름은 내부적으로 색인으로 변환된다.

In [80]:
obj = pd.Series(range(3), index = ['a','b','c'])
index = obj.index

In [84]:
print('''
index : {0}
index[1:] : {1}'''.format(index, index[1:]))


index : Index(['a', 'b', 'c'], dtype='object')
index[1:] : Index(['b', 'c'], dtype='object')


> 색인 객체는 변경이 불가능하다.

In [85]:
index[1] = 'd'

TypeError: Index does not support mutable operations

> 그러므로 자료구조 사이에서 안전하게 공유될 수 있다.

In [90]:
labels = pd.Index(np.arange(3))
labels

Int64Index([0, 1, 2], dtype='int64')

In [92]:
obj2 = pd.Series([1.5,-2.5,0], index = labels)
obj2

0    1.5
1   -2.5
2    0.0
dtype: float64

In [93]:
obj2.index is labels

True

#### 색인 메서드와 속성

<details>
<summary>색인 메서드와 속성</summary>
<div markdown="1">

|메서드|설명|
|:--|:--|
|append|추가적인 색인 객체를 덧붙여 새로운 색인을 반환한다.|
|difference|색인의 차집합을 반환한다.|
|intersection|색인의 교집합을 반환한다.|
|union|색인의 합집합을 반환한다.|
|isin|색인이 넘겨받은 색인에 존재하는지 알려주는 불리언 배열을 반환한다.|
|delete|i 위치의 색인이 삭제된 새로운 색인을 반환한다.|
|drop|넘겨받은 값이 삭제된 새로운 색인을 반환한다.|
|insert|i 위치에 색인이 추가된 새로운 색인을 반환한다.|
|is_monotonic|색인이 단조성을 가진다면 True를 반환한다.|
|is_unique|중복되는 색인이 없다면 True를 반환한다.|
|unique|색인에서 중복되는 요소를 제거하고 유일한 값만 반환한다.|
    
</div>
</details>

## 5.2 핵심 기능

### 5.2.1 재색인
- reindex : 새로운 색인에 맞도록 객체를 `새로 생성`한다.

In [95]:
obj = pd.Series([4.5,7.2,-5.3,3.6], index = ['d','b','a','c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

> Series 객체에 대해 reindex를 호출하면 데이터를 새로운 색인에 맞게 재배열함.  
> 존재하지 않는 색인값이 있다면 NaN을 추가함.

In [96]:
obj2 = obj.reindex(['a','b','c','d','e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

> 시계열 같은 순차적인 데이터를 재색인할 때 값을 보간하거나 채워 넣어야 할 경우가 있다.  
> 이는 method 옵션을 이용해서 해결할 수 있다.  
> ffill 메서드는 누락된 값을 직전의 값으로 채워 넣는 방법을 의미한다.

In [98]:
obj3 = pd.Series(['blue','purple','yellow'], index = [0,2,4])
obj3

0      blue
2    purple
4    yellow
dtype: object

In [105]:
obj3.reindex(range(6), method = "ffill")

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

>  DataFrame에 대한 reindex는 로우(색인), 컬럼 또는 둘 다 변경이 가능함.  
> 그냥 순서만 전달하면 로우가 재색인된다.

In [109]:
frame = pd.DataFrame(np.arange(9).reshape((3,3)),
                     index = ['a','c','d'],
                     columns = ['Ohio','Texas','California'])
frame

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [110]:
frame2 = frame.reindex(['a','b','c','d'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


> 컬럼은 columns 예약어를 사용해서 재색인할 수 있다.

In [111]:
states = ['Texas', 'Utah', 'California']

frame.reindex(columns = states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


#### reindex의 인자

<details>
<summary>재색인 함수 인자</summary>
<div markdown="1">

|인자|설명|
|:--|:--|
|index|색인으로 사용할 새로운 순서. Index 인스턴스나 다른 순차적인 자료구조가 사용 가능하다. Index는 복사가 이루어지지 않고 그대로 사용된다.|
|method|채움 메서드. ffill은 직전 값을 채워 넣고 bfill은 다음 값을 채워 넣는다.|
|fill_value|재색인 과정 중에 새롭게 나타나는 비어 있는 데이터를 채우기 위한 값|
|limit|전/후 보간 시에 사용할 최대 갭 크기 (채워넣을 원소의 수)|
|tolerance|전/후 보간 시에 사용할 최대 갭 크기 (값의 차이)|
|level|MultiIndex의 단계(level)에 단순 색인을 맞춘다. 그렇지 않으면 MultiIndex의 하위집합에 맞춘다.|
|copy|True인 경우 새로운 색인이 이전 색인과 동일하더라도 데이터를 복사한다. False인 경우 새로운 색인이 이전 색인과 동일할 경우 복사하지 않는다.|   
    
</div>
</details>

### 5.2.2. 하나의 로우나 컬럼 삭제하기

- drop 메서드를 사용하면 선택한 값들이 삭제된 `새로운 객체` 를 얻을 수 있다.

In [113]:
obj = pd.Series(np.arange(5.), index = ['a','b','c','d','e'])

obj.drop(['d','c'])

a    0.0
b    1.0
e    4.0
dtype: float64

> DataFrame에서는 로우와 컬럼 모두에서 값을 삭제할 수 있다.

In [123]:
data = pd.DataFrame(np.arange(16).reshape((4,4)),
                    index = ['Ohio','Colorado','Utah','New York'],
                    columns = ['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


> drop 함수 인자에 로우 이름을 넘기면 해당 로우 (axis 0)의 값을 모두 삭제한다.

In [124]:
data.drop(['Colorado','Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


> 컬럼의 값을 삭제할 때는 axis = 1 또는 axis = 'columns'를 인자로 넘겨주면 된다.

In [125]:
data.drop('two', axis = 1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [126]:
data.drop(['two','four'], axis = 'columns')

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


> inplace 옵션을 사용하면 새로운 객체를 반환하는 대신 원본 객체를 변경한다.

In [127]:
data.drop(['two','four'], axis = 'columns', inplace = True)

In [128]:
data

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


### 5.2.3 색인하기, 선택하기, 거르기

> Series의 색인은 NumPy 배열의 색인과 유사하게 동작하지만 정수가 아니어도 된다는 점이 다르다.

In [130]:
obj = pd.Series(np.arange(4.), index = ['a','b','c','d'])

In [132]:
obj[2:4]

c    2.0
d    3.0
dtype: float64

In [133]:
obj[['b','c']]

b    1.0
c    2.0
dtype: float64

In [134]:
obj[[1,3]]

b    1.0
d    3.0
dtype: float64

In [135]:
obj[obj<2]

a    0.0
b    1.0
dtype: float64

In [137]:
obj['b':'c'] # 라벨 이름으로 슬라이싱하면 시작점과 끝점을 포함한다는 것이 일반 파이썬에서의 슬라이싱과 다른 점 

b    1.0
c    2.0
dtype: float64

> 색인으로 DataFrame에서 하나 이상의 컬럼 값을 가져올 수 있다.

In [139]:
data = pd.DataFrame(np.arange(16).reshape((4,4)), 
                    index = ['Ohio','Colorado','Utah','New York'],
                    columns = ['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [140]:
data[['three','one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


> 슬라이싱으로 로우를 선택하거나 불리언 배열로 로우를 선택할 수 있다.

In [141]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [142]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


#### loc과 iloc으로 선택하기
- loc과 iloc는 DataFrame의 `로우`에 대해 `라벨`로 색인할 수 있게 해주는 특수한 색인 필드
- 이 방법을 이용하면 NumPy와 비슷한 방식으로 추가적으로 `축의 라벨`을 사용하여 DataFrame의 로우와 컬럼을 선택할 수 있다.
- `축 이름`을 선택할 때는 `loc`을, `정수 색인`으로 선택할 때는 `iloc`을 사용한다.

In [156]:
data.loc[['Colorado','Utah'], ['two','three']]

,two,three
Colorado,5,6
Utah,9,10


In [158]:
data.iloc[[1,2], [3,0,1]]

,four,one,two
Colorado,7,4,5
Utah,11,8,9


> 이 두 함수는 슬라이스도 지원할 뿐더러 단일 라벵리나 라벨 리스트도 지원한다.

In [160]:
data.loc[:"Utah",:"two"]

,one,two
Ohio,0,1
Colorado,4,5
Utah,8,9


In [163]:
data.iloc[:,:3][data.three > 5]

,one,two,three
Colorado,4,5,6
Utah,8,9,10
New York,12,13,14


#### pandas 객체에서 데이터를 선택하고 재배열하는 방법

<details>
<summary>DataFrame의 값 선택하기</summary>
<div markdown="1">

|방식|설명|
|:--|:--|
|df[val]|DataFrame에서 하나의 컬럼 또는 여러 컬럼을 선택한다. 편의를 위해 불리언 배열, 슬라이스, 불리언 DataFrame을 사용할 수 있다.|
|df.loc[val]|DataFrame에서 라벨값으로 `로우`의 부분집합을 선택한다.|
|df.loc[:,val]|DataFrame에서 라벨값으로 `컬럼`의 부분집합을 선택한다.|
|df.loc[val1,val2]|DataFrame에서 라벨값으로 `로우와 컬럼`의 부분집합을 선택한다.|
|df.iloc[where]|DataFrame에서 정수 색인으로 `로우`의 부분집합을 선택한다.|
|df.iloc[:, where]|DataFrame에서 정수 색인으로 `컬럼`의 부분집합을 선택한다.|
|df.iloc[where_i,where_j]|DataFrame에서 정수 색인으로 `로우와 컬럼`의 부분집합을 선택한다.|
|df.at[label_i, label_j]|로우와 컬럼의 라벨로 `단일 값`을 선택한다.|
|df.iat[i,j]|로우와 컬럼의 정수 색인으로 `단일 값`을 선택한다.|
|reindex 메서드|하나 이상의 축을 새로운 색인으로 맞춘다.|
|get_value, set_value 메서드|로우와 컬럼 이름으로 DataFrame의 값을 선택한다.|
    
</div>
</details>